# Week 9: Local Motion Planning

### Topics Covered

- Sampling-based Planners (RRT/RRT*); Search-based Planners (State Lattice); Generating a collision-free path

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the key concepts
2. Implement algorithms
3. Apply techniques to real-world problems

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Polygon
from matplotlib.collections import LineCollection
import time

# Set random seed for reproducibility
np.random.seed(42)

## 1. Motion Planning Fundamentals

Local motion planning generates **collision-free trajectories** from the current position to a nearby goal, accounting for obstacles and vehicle dynamics.

### Planning Hierarchy

1. **Mission Planning**: Route on road network (A*, Dijkstra)
2. **Behavior Planning**: High-level decisions (FSM, behavior trees)
3. **Local Motion Planning**: Collision-free trajectory generation ← **This week**
4. **Control**: Execute trajectory (PID, MPC)

### Configuration Space (C-space)

**Definition:** The space of all possible robot configurations.

For a 2D vehicle:
- **Work Space**: Physical (x, y) space with obstacles
- **Configuration Space**: (x, y, θ) - position + heading
- **State Space**: (x, y, θ, v) - includes velocity

**C-space Obstacle:** Set of configurations where robot collides with obstacles.

### Motion Planning Problem

**Given:**
- Start configuration $q_{start}$
- Goal configuration $q_{goal}$
- Obstacle set $\mathcal{O}$ in C-space

**Find:**
- Path $\pi: [0, 1] \rightarrow \mathcal{C}_{free}$
- Where $\pi(0) = q_{start}$, $\pi(1) = q_{goal}$
- And $\pi(s) \notin \mathcal{O}$ for all $s \in [0, 1]$

**Objective:** Minimize cost (e.g., path length, time, energy)

### Planning Approaches

| Approach | Method | Completeness | Optimality |
|----------|--------|--------------|------------|
| **Sampling-based** | RRT, RRT* | Probabilistic | RRT*: Asymptotic |
| **Search-based** | A*, State Lattice | Resolution | Yes (discretization) |
| **Optimization** | Trajectory optimization | Local | Local minimum |

### Challenges in Autonomous Driving

1. **High-dimensional state space**: (x, y, θ, v, a)
2. **Nonholonomic constraints**: Can't move sideways
3. **Dynamic obstacles**: Other vehicles, pedestrians
4. **Real-time requirements**: < 100ms planning cycles
5. **Safety**: Must guarantee collision avoidance

## 4. State Lattice Planning

State lattice planning uses a **discrete graph** of kinematically-feasible motions precomputed for the vehicle.

### Key Idea

Instead of random sampling, use a **library of motion primitives** that respect vehicle constraints (e.g., turning radius, acceleration limits).

### Motion Primitives

Precomputed trajectories from a reference state:
- Straight
- Left turn (various radii)
- Right turn (various radii)
- Lane change left
- Lane change right

### State Lattice Construction

1. **Discretize state space**: (x, y, θ) grid
2. **For each state, apply motion primitives** to reach neighboring states
3. **Build graph** of reachable states
4. **Search graph** using A* or Dijkstra

### Advantages

- **Kinematically feasible**: Paths respect vehicle dynamics
- **Optimal on lattice**: A* guarantees shortest path on graph
- **Fast**: Graph search is efficient
- **Predictable**: Deterministic results

### Disadvantages

- **Resolution completeness**: Only finds solutions on the lattice
- **Memory intensive**: Must store all primitives
- **Requires tuning**: Lattice resolution vs. computation

### Applications

- **Highway driving**: Structured environments with lanes
- **Parking**: Low-speed maneuvering
- **Urban navigation**: Road-constrained motion

## Exercises

### Exercise 1: Implement RRT*

Extend the RRT implementation to add the rewiring step for path optimization.

**Tasks:**
1. Implement the `Near()` function to find nodes within radius
2. Add "Choose Best Parent" logic
3. Add "Rewire Tree" logic
4. Compare path quality (cost) between RRT and RRT*
5. Plot cost vs. iterations to show convergence

**Hint:** Use shrinking radius $r = \gamma (\log n / n)^{1/2}$ where $\gamma \approx 10-20$

In [ ]:
# Exercise 3 - Template

class DynamicObstacle:
    """Moving obstacle"""
    def __init__(self, x0, y0, vx, vy, radius):
        self.x0 = x0
        self.y0 = y0
        self.vx = vx
        self.vy = vy
        self.radius = radius
    
    def position_at_time(self, t):
        """Return position at time t"""
        return (self.x0 + self.vx * t, self.y0 + self.vy * t)

# TODO: Extend the RRT class from Section 2 to handle dynamic obstacles
# 
# Hints:
# 1. Add time as a state variable: RRTNode should store (x, y, t)
# 2. Modify collision checking to check space-time collision
# 3. When steering, also advance time proportional to distance traveled
# 4. Check if path intersects with moving obstacle trajectory
#
# Example approach:
# class DynamicRRT(RRT):  # Extend RRT from above
#     def __init__(self, environment, start, goal, dynamic_obstacles, **kwargs):
#         super().__init__(environment, start, goal, **kwargs)
#         self.dynamic_obstacles = dynamic_obstacles
#     
#     def is_collision_at_time(self, x, y, t):
#         """Check collision with moving obstacles at time t"""
#         # Check static obstacles (from parent class)
#         if self.env.is_collision(x, y):
#             return True
#         
#         # Check dynamic obstacles
#         for obs in self.dynamic_obstacles:
#             obs_x, obs_y = obs.position_at_time(t)
#             dist = np.sqrt((x - obs_x)**2 + (y - obs_y)**2)
#             if dist < obs.radius:
#                 return True
#         return False

# Your implementation here

## References and Additional Resources

### Core Textbooks

1. **Planning Algorithms** by LaValle (2006)
   - Chapters 5-6: Sampling-based Planning
   - Comprehensive coverage of RRT, RRT*
   - Free online: http://planning.cs.uiuc.edu/

2. **Robotics: Modelling, Planning and Control** by Siciliano et al. (2009)
   - Chapter 6: Motion Planning

3. **Principles of Robot Motion** by Choset et al. (2005)
   - Path planning algorithms
   - Configuration space concepts

### Key Papers

1. **RRT**
   - LaValle (1998) - "Rapidly-Exploring Random Trees: A New Tool for Path Planning"
   - Original RRT paper

2. **RRT***
   - Karaman & Frazzoli (2011) - "Sampling-based Algorithms for Optimal Motion Planning"
   - Proves asymptotic optimality

3. **State Lattice**
   - Pivtoraiko & Kelly (2005) - "Generating Near Minimal Spanning Control Sets for Constrained Motion Planning"
   - McNaughton et al. (2011) - "Motion Planning for Autonomous Driving with a Conformal Spatiotemporal Lattice"

4. **Dynamic Environments**
   - Ferguson et al. (2008) - "Replanning with RRTs"
   - Anytime Dynamic A*

### Algorithms & Variants

1. **RRT Variants**
   - RRT-Connect: Bidirectional RRT
   - RRT*-Smart: Improved sampling
   - Informed RRT*: Ellipsoidal sampling
   - Kinodynamic-RRT: Respects dynamics

2. **Other Sampling Methods**
   - PRM (Probabilistic Roadmap)
   - EST (Expansive Space Trees)
   - SBL (Single-Query Bidirectional Lazy Planner)

3. **Search-Based Methods**
   - Field D*: Interpolated paths
   - Hybrid A*: Continuous state space
   - Theta*: Any-angle planning

### Software & Libraries

1. **OMPL (Open Motion Planning Library)**
   - C++ library with Python bindings
   - Implements RRT, RRT*, PRM, etc.
   - https://ompl.kavrakilab.org/

2. **Python Robotics**
   - Educational implementations
   - https://github.com/AtsushiSakai/PythonRobotics

3. **MoveIt** (ROS)
   - Motion planning framework
   - Industrial manipulation and mobile robots

### Applications in Autonomous Vehicles

1. **Highway Driving**
   - Lattice planners for lane keeping/changing
   - Polynomial trajectory optimization

2. **Urban Driving**
   - Hybrid approaches (lattice + optimization)
   - Interaction-aware planning

3. **Parking**
   - RRT for maneuvering in tight spaces
   - Reeds-Shepp curves

### Advanced Topics

1. **Kinodynamic Planning**
   - Incorporate acceleration/velocity limits
   - Respect vehicle dynamics

2. **Uncertainty**
   - Chance-constrained RRT
   - Robust trajectory optimization

3. **Learning-Based**
   - Neural Motion Planning
   - Learning sampling distributions
   - Imitation learning for primitives

4. **Multi-Query Planning**
   - PRM for multiple start-goal queries
   - Roadmap maintenance

### Practical Considerations

1. **Real-Time Planning**
   - Anytime algorithms
   - Incremental replanning
   - Computing budgets (< 100ms)

2. **Safety**
   - Guaranteed collision avoidance
   - Safe corridors
   - Emergency maneuvers

3. **Smoothness**
   - Path smoothing post-processing
   - Bezier/spline fitting
   - Jerk minimization

---

## Summary

This notebook covered local motion planning for autonomous vehicles:

**Key Concepts:**
- Configuration space and collision detection
- RRT for probabilistic complete planning
- RRT* for asymptotically optimal paths
- State lattice for kinematically-feasible planning

**Practical Skills:**
- Implementing sampling-based planners
- Building collision-free paths
- Understanding trade-offs (completeness vs. optimality vs. speed)
- Handling complex environments with obstacles

**Next Steps:**
1. Study trajectory optimization (MPC, numerical optimization)
2. Learn about prediction-aware planning
3. Explore learning-based motion planning
4. Implement real-time replanning strategies